<a href="https://colab.research.google.com/github/Toadoum/STEM-Away-Job/blob/master/Training_skills_extraction_and_labelling_STEM_Away.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##  Latent Dirichlet Allocation (LDA)

In [7]:
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=6277565c4904d98cdfae500cc3788daa59ff27f6f97568a64dcb8548456dc38e
  Stored in directory: /root/.cache/pip/wheels/90/61/ec/9dbe9efc3acf9c4e37ba70fbbcc3f3a0ebd121060aa593181a
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=7bb0be31d4d9520ecf04d8422549211075f737c3d12ca84b9d8e556413e0f4fe
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built pyLDAvis sklearn


We use Latent Dirichlet Allocation (LDA) to extract the most common topics from our data, and then use the top words from each topic as our skill_keywords.

In [9]:
df = pd.read_csv('/content/drive/My Drive/STEMAway/df_description_processed.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1006 entries, 0 to 1005
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Description         1006 non-null   object
 1   lower_description   1006 non-null   object
 2   word_tokenized      1006 non-null   object
 3   sentence_tokenized  1006 non-null   object
 4   word_count          1006 non-null   int64 
 5   sentence_count      1006 non-null   int64 
 6   clean_words         1006 non-null   object
 7   clean_stemmed       1006 non-null   object
 8   clean_lemmed        1006 non-null   object
dtypes: int64(2), object(7)
memory usage: 70.9+ KB


In [10]:
# create the transform
vectorizer = CountVectorizer(stop_words = 'english', max_df = 0.75)
# tokenize and build vocab
bag_of_words = vectorizer.fit_transform(df.clean_lemmed)
# summarize
sum_words = bag_of_words.sum(axis=0) 
words_freq = [(word, sum_words[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
words_freq[:10]

[('model', 1942),
 ('ml', 1612),
 ('development', 1475),
 ('ai', 1437),
 ('business', 1431),
 ('skill', 1428),
 ('technology', 1419),
 ('product', 1416),
 ('research', 1382),
 ('engineering', 1371)]

In [11]:
# 5379 documents represented as a 14,409 dimensional vector (14,409 words)
bag_of_words

<1006x15555 sparse matrix of type '<class 'numpy.int64'>'
	with 194431 stored elements in Compressed Sparse Row format>

In [12]:
LDA = LatentDirichletAllocation(n_components=5, random_state=42)
LDA.fit(bag_of_words)

LatentDirichletAllocation(n_components=5, random_state=42)

In [13]:
first_topic = LDA.components_[0]

In [14]:
top_topic_words = first_topic.argsort()[-10:]
top_topic_words

array([12981, 15289,  4967, 13890,  4198,  2396, 10867, 12815,  8972,
        9026])

In [15]:
for i in top_topic_words:
    print(vectorizer.get_feature_names()[i])

solution
working
engineering
technology
development
business
product
skill
ml
model


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [16]:
for i,topic in enumerate(LDA.components_):
    print(f'Top 10 words for topic #{i}:')
    print([vectorizer.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for topic #0:
['solution', 'working', 'engineering', 'technology', 'development', 'business', 'product', 'skill', 'ml', 'model']


Top 10 words for topic #1:
['support', 'skill', 'company', 'australia', 'abbvie', 'project', 'group', 'zealand', 'client', 'new']


Top 10 words for topic #2:
['year', 'computer', 'development', 'application', 'model', 'company', 'opportunity', 'business', 'product', 'technology']


Top 10 words for topic #3:
['technology', 'opportunity', 'engineer', 'development', 'company', 'engineering', 'business', 'solution', 'software', 'ai']


Top 10 words for topic #4:
['employment', 'candidate', 'position', 'development', 'student', 'computer', 'university', 'ai', 'application', 'research']




In [17]:
topic_values = LDA.transform(bag_of_words)
topic_values.shape


(1006, 5)

In [18]:
df['Topic'] = topic_values.argmax(axis=1)

In [19]:
df.head()

,Description,lower_description,word_tokenized,sentence_tokenized,word_count,sentence_count,clean_words,clean_stemmed,clean_lemmed,Topic
0,"Job DescriptionBuild software, save the world!...","job descriptionbuild software, save the world!...","['job', 'descriptionbuild', 'software', 'save'...","['Job DescriptionBuild software, save the worl...",247,1,"['job', 'descriptionbuild', 'software', 'save'...","['job', 'descriptionbuild', 'softwar', 'save',...","['job', 'descriptionbuild', 'software', 'save'...",3
1,About The CompanyThis startup company has rece...,about the companythis startup company has rece...,"['about', 'the', 'companythis', 'startup', 'co...","[""About The CompanyThis startup company has re...",186,1,"['companythis', 'startup', 'company', 'receive...","['companythi', 'startup', 'compani', 'receiv',...","['companythis', 'startup', 'company', 'receive...",0
2,"Anduril is a defence technology company, bring...","anduril is a defence technology company, bring...","['anduril', 'is', 'a', 'defence', 'technology'...","['Anduril is a defence technology company, bri...",301,7,"['anduril', 'defence', 'technology', 'company'...","['anduril', 'defenc', 'technolog', 'compani', ...","['anduril', 'defence', 'technology', 'company'...",0
3,About RoktRokt is the global leader in eCommer...,about roktrokt is the global leader in ecommer...,"['about', 'roktrokt', 'is', 'the', 'global', '...",['About RoktRokt is the global leader in eComm...,1237,36,"['roktrokt', 'global', 'leader', 'ecommerce', ...","['roktrokt', 'global', 'leader', 'ecommerc', '...","['roktrokt', 'global', 'leader', 'ecommerce', ...",3
4,"Canva’s Commitment and MissionAt Canva, we cel...","canva’s commitment and missionat canva, we cel...","['canva', 's', 'commitment', 'and', 'missionat...","['Canva’s Commitment and MissionAt Canva, we c...",244,6,"['canva', 'commitment', 'missionat', 'canva', ...","['canva', 'commit', 'missionat', 'canva', 'cel...","['canva', 'commitment', 'missionat', 'canva', ...",0


In [8]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from wordcloud import WordCloud
import re

import matplotlib.pyplot as plt 
import seaborn as sns
from wordcloud import WordCloud
import pyLDAvis
sns.set()

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import gensim


# import nltk
# from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
# from nltk.stem.porter import PorterStemmer
# from nltk.stem.wordnet import WordNetLemmatizer
# from nltk.corpus import stopwords

/usr/local/lib/python3.8/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable
/usr/local/lib/python3.8/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Mapping
/usr/local/lib/python3.8/dist-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [20]:
import pandas as pd
import pickle

# chunks were taken from regex of POS tags located on google colab
chunks1 = pickle.load( open('/content/drive/My Drive/STEMAway/chunks_1.pickle', "rb" ) )
chunks2 = pickle.load( open('/content/drive/My Drive/STEMAway/chunks_2.pickle', "rb" ) )
chunks3 = pickle.load( open('/content/drive/My Drive/STEMAway/chunks_3.pickle', "rb" ) )
chunks4 = pickle.load( open('/content/drive/My Drive/STEMAway/chunks_4.pickle', "rb" ) )

# Sample size is 10% and will be labeled accordingly
# perhaps a sample of a sample can be used depends on NN model
print('Length:', len(chunks1), 'Sample Size:', len(chunks1) * .10)
print('Length:', len(chunks2), 'Sample Size:', len(chunks2) * .10) 
print('Length:', len(chunks3), 'Sample Size:', len(chunks3) * .10)
print('Length:', len(chunks4), 'Sample Size:', len(chunks4) * .10)

def training_set(chunks):
    '''creates a dataframe that easily parsed with the chunks data '''
    df = pd.DataFrame(chunks)    
    df.fillna('X', inplace = True)
    
    train = []
    for row in df.values:
        phrase = ''
        for tup in row:
            # needs a space at the end for seperation
            phrase += tup[0] + ' '
        phrase = ''.join(phrase)
        # could use padding tages but encoder method will provide during 
        # tokenizing/embeddings; X can replace paddding for now
        train.append( phrase.replace('X', '').strip())

    df['phrase'] = train

    # only returns 10% of each dataframe to be used 
    return df.phrase.sample(frac = 0.1)

# one training corpus with 10% of each POS regex identification
training = pd.concat([training_set(chunks1),
                      training_set(chunks2), 
                      training_set(chunks3),
                      training_set(chunks4)], 
                        ignore_index = True )

training.to_csv('train_skills.csv')
print("'train_skills.csv' has been created")

Length: 128710 Sample Size: 12871.0
Length: 130592 Sample Size: 13059.2
Length: 65436 Sample Size: 6543.6
Length: 6478 Sample Size: 647.8000000000001
'train_skills.csv' has been created


In [21]:
# Extract the top words for each topic
top_words = []
for index, topic in enumerate(LDA.components_):
    top_words.extend([vectorizer.get_feature_names()[i] for i in topic.argsort()[-10:]])

# Remove duplicates
skill_keywords = list(set(top_words))


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [22]:
skill_keywords

['new',
 'student',
 'year',
 'skill',
 'solution',
 'business',
 'ml',
 'software',
 'university',
 'employment',
 'project',
 'ai',
 'company',
 'group',
 'abbvie',
 'technology',
 'engineer',
 'model',
 'candidate',
 'support',
 'product',
 'opportunity',
 'australia',
 'development',
 'engineering',
 'client',
 'application',
 'computer',
 'zealand',
 'position',
 'research',
 'working']

After doing Latent Dirichlet Allocation (LDA), we can extract the top words from each topic as our skill_keywords.

In [24]:
import pandas as pd

# Load the dataset
df = pd.read_csv('train_skills.csv')

# Create an empty column called "skill"
df['skill'] = 0

# Define a list of keywords that represent skills
skill_keywords = skill_keywords
# Iterate over the phrases in the dataset
for i, row in df.iterrows():
    phrase = row['phrase']
    skill = 0
    # Check if any of the keywords are present in the phrase
    for keyword in skill_keywords:
        if keyword in phrase:
            skill = 1
            break
    # Update the "skill" column with the value of 1 or 0
    df.at[i, 'skill'] = skill

# Save the updated dataset to a new CSV file
df.to_csv('train_skills_labeled.csv', index=False)
print("'train_skills_labeled.csv' has been created")


'train_skills_labeled.csv' has been created
